# Run SageMaker tuning jobs
This notebook uses SageMaker tuning jobs for running hyperparameter tuning jobs

In [20]:
import os
import pickle
import random

import boto3
import numpy as np
import pandas as pd
import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

In [21]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()

In [22]:
hyperparameter_ranges = {
    "learning_rate": ContinuousParameter(1e-2, 1),
    'batch_size': CategoricalParameter([64, 128, 256]),
    't_0': CategoricalParameter([1, 10, 50]),
    't_mult': CategoricalParameter([1, 2])
}

In [23]:
objective_metric_name = "Validation QWK"
objective_type = "Maximize"
metric_definitions = [{"Name": "Validation QWK", "Regex": "Best average val QWK over 5-folds: ([0-9\\.]+)"}]

In [25]:
train_instance_type = "ml.c5.4xlarge"

estimator_parameters = {
    "entry_point": "train.py",
    "source_dir": "trainer",
    "instance_type": train_instance_type,
    "instance_count": 1,
    "hyperparameters": {'epochs': 4000, 'filename': ''},
    "role": role,
    "base_job_name": "dupsko",
    "framework_version": "1.5",
    "py_version": "py3",
}

In [26]:
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix='data/embeddings/avg/en/')['Contents']

In [41]:
for f in contents:
    abspath = f['Key']
    path = abspath.split('/')[0]
    filename = abspath.split('/')[-1]
    if '.pt' not in filename:
        continue
    
    print(abspath)
    print(filename)
    
    model = filename.split('.')[0]
    inputs = {"train": f's3://{bucket}/{abspath}'}
    
    # For logging
    estimator_parameters['hyperparameters']['filename'] = filename
    estimator_parameters['base_job_name'] = filename
    
    
    estimator = PyTorch(**estimator_parameters)
#     estimator.fit(inputs)
    
    tuner = HyperparameterTuner(
        estimator,
        objective_metric_name,
        hyperparameter_ranges,
        metric_definitions,
        max_jobs=10,
        max_parallel_jobs=4,
        objective_type=objective_type,
        base_tuning_job_name=f'{model}-5-labels',
    )
    
    tuner.fit(inputs)

data/embeddings/avg/en/en-allenai-longformer-base-4096.pt
en-allenai-longformer-base-4096.pt
..........................................................................................................................................................................................................................................................................!
data/embeddings/avg/en/en-allenai-longformer-large-4096.pt
en-allenai-longformer-large-4096.pt
............................................................................................................................................

KeyboardInterrupt: 